## Note - This example code uses a SQL Server instance running on a docker Linux container.  If you have created a container using the BuildContainer notebook, use these scripts as is.  If you are running on a native instance, modify the scripts according the comments.  

## Step 1 - Connect to your SQL Server instance using Azure Data Studio.

In [16]:
/* If you created a SQL container in the previous step, you can use this script as is.
   If you are using a different instance, change the FILENAME paths to be valid.
   Change the database and object names if you must.  */
USE master
GO
DROP DATABASE IF EXISTS whitesox
GO
CREATE DATABASE whitesox
ON PRIMARY
(NAME = N'whitesox_primary', FILENAME = '/var/opt/mssql/data/whitesox.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'whitesox_Log', FILENAME = '/var/opt/mssql/data/whitesox_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO

ALTER DATABASE whitesox
ADD FILEGROUP PVS_FG;
GO
ALTER DATABASE whitesox
ADD FILE
(
    NAME = whitesox_pvs,
    FILENAME = '/var/opt/mssql/data/whitesox_pvs.ndf',
    SIZE = 1GB,
    FILEGROWTH = 1MB
)
TO FILEGROUP PVS_FG;
GO

ALTER DATABASE whitesox SET RECOVERY SIMPLE
GO
USE whitesox
GO
DROP TABLE IF EXISTS letsgowhitesox
GO
CREATE TABLE letsgowhitesox (rowid int primary key clustered, rowtext char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 100000)
BEGIN
	INSERT INTO letsgowhitesox VALUES (@x, 'Hoping for baseball with fans in 2021!')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF

USE master;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'whitesox';



Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:01:10.596

name,is_accelerated_database_recovery_on
whitesox,0


## Step 2 - Delete all rows for the table in an open transaction.

In [17]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
USE whitesox
GO
BEGIN TRAN
DELETE from letsgowhitesox
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:35.812

## Step 3 - Check transaction log usage.
### Because there is an open transaction, the log file will contain space that cannot be reused.  After checking log usage, run a CHECKPOINT and see if log space is freed.

In [18]:
USE whitesox;

SELECT * FROM sys.dm_db_log_space_usage

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage

GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:02.318

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,4223787008,78.674286,971538432


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,4223864832,78.675735,1000116224


## Step 4 - How long does it take to rollback the deletes?
### Rollback the transaction and check the duration.

In [19]:
USE whitesox;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:49.552

## Step 5 - Enable Accelerated Database Recovery

In [20]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.157

## Step 6 - Delete all of the rows again within a transaction

In [21]:
USE whitesox
GO

BEGIN TRAN
DELETE from letsgowhitesox;

GO

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:15.805

## Step 7 - Check log space used before and after a CHECKPOINT
### Notice log space usage is low even before the CHECKPOINT

In [22]:
USE whitesox;

SELECT d.name, d.is_accelerated_database_recovery_on, p.persistent_version_store_size_kb 
FROM sys.databases d
INNER JOIN sys.dm_tran_persistent_version_store_stats p
ON p.database_id = d.database_id
WHERE is_accelerated_database_recovery_on = 1;

SELECT * FROM sys.dm_db_log_space_usage
GO

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage
GO


(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:02.893

name,is_accelerated_database_recovery_on,persistent_version_store_size_kb
whitesox,1,0


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,1392209920,25.93197,1101860864


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,385892352,7.187816,81920


## Step 8 - How fast is a ROLLBACK?
### Is ROLLBACK any faster with ADR enabled?

In [23]:
USE whitesox;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:00.003